# Initial Setup

This Notebook describes the various setup options for using the potentials package that users can control.  While none of these steps are required, they provide users more control over how the package works.  

1. Check/set local paths
2. Change default database settings
3. Set default list of installed KIM models
4. Create local copy of database

Most of these features are controlled by the Settings class.

In [1]:
import potentials
from potentials import settings

## 1. Check/set local paths

### 1.1. Check/set settings file location

The potentials package handles user-defined settings by creating a settings.json file.  This file is located in the user's home directory. The current directory and the settings file in it can be viewed using the __directory__ and __filename__ attributes.

In [2]:
print(settings.directory)
print(settings.filename)

C:\Users\lmh1\.NISTpotentials
C:\Users\lmh1\.NISTpotentials\settings.json


The directory can be changed using __set_directory()__, if you wish.  This can be useful if multiple environments with different home directories are to share the same settings.  __NOTE__: the directory can only be changed if no other settings have been modified!

In [3]:
#settings.set_directory('C:/Users/lmh1/test')

The directory and filename attributes will reflect this change.  However, the original default values are still retained in the __defaultdirectory__ and __defaultfilename__ attributes.

In [3]:
print(settings.directory)
print(settings.filename)
print(settings.defaultdirectory)
print(settings.defaultfilename)

C:\Users\lmh1\.NISTpotentials
C:\Users\lmh1\.NISTpotentials\settings.json
C:\Users\lmh1\.NISTpotentials
C:\Users\lmh1\.NISTpotentials\settings.json


The settings directory can be reverted back to the default by using __unset_directory()__.

In [4]:
settings.unset_directory()
print(settings.directory)

No settings directory set, still using default C:\Users\lmh1\.NISTpotentials
C:\Users\lmh1\.NISTpotentials


### 1.2. Check/set local library directory

The location of the directory to use for the local copy of the database (i.e. the library) by default is within the settings directory.  The location be checked using the __library_directory__ attribute and changed with the __set_library_directory()__ and __unset_library_directory()__ methods.  Changing the library directory allows for the local records to be stored in a more accessible location than the default path.

In [5]:
settings.library_directory

WindowsPath('C:/Users/lmh1/Documents/library')

In [6]:
settings.set_library_directory('C:/Users/lmh1/Documents/library')

Library directory already set to C:\Users\lmh1\Documents\library
Overwrite? (yes or no): no


In [7]:
settings.library_directory

WindowsPath('C:/Users/lmh1/Documents/library')

In [8]:
settings.unset_library_directory()

Remove library directory C:\Users\lmh1\Documents\library and reset to C:\Users\lmh1\.NISTpotentials\library?
Delete settings? (must type yes): no


## 2. Change default database settings

The default values of some optional parameters associated with how the Database class operates can also be changed.  Setting these may be useful depending on how you primarily intend to use the potentials package.

- __remote__ (*bool*) indicates if the database will search for records from the remote database, i.e. potentials.nist.gov.
- __local__ (*bool*) indicates if the database will search for records from the local path database.
- __pot_dir_style__ (*str*) indicates where the associated parameter files for LAMMPS potentials are expected to be found when running simulations.  This setting alters where the LAMMPS commands point to and where parameter files are copied/downloaded to.
    - "working" means parameter files are expected in the working directory.
    - "id" means parameter files are expected in a subdirectory of the working directory matching the LAMMPS potential's id.
    - "local" means parameter files are expected in the local path database.
    


In [9]:
print('Default Database settings are:')
print(settings.local)
print(settings.remote)
print(settings.pot_dir_style)

Default Database settings are:
True
False
working


Each of these values has a corresponding "__set__" method that allows for the default value to be changed.  Note that the default values can always be overridden when initializing and using the Database class.

In [5]:
# Local and remote change where the files are accessed from
settings.set_local(True)
settings.set_remote(False)

# pot_dir_style changes where
#settings.set_pot_dir_style('working')
settings.set_pot_dir_style('id')
#settings.set_pot_dir_style('local')

# Initialize default database that uses the values set above and loads LAMMPS potentials
db = potentials.Database(load='lammps_potentials', verbose=True)

# Show pot dir for the first loaded LAMMPS potential
print()
print(repr(db.lammps_potentials[0].pot_dir))

Loaded 401 local LAMMPS potentials
Building lammps potentials for kim models
Found 450 records in local library
Loaded 302 installed KIM LAMMPS potentials

''


## 3. Set default list of installed KIM models

*Added version 0.3.0* 

In order for models from openKIM to be included alongside native LAMMPS potentials, the potentials package needs to know which kim models to include.  The list can be defined during Database initialization, or default options can be saved using settings for convenience.  The two related settings options are

- Explicitly saving a list of kim models using __set_kim_models()__, which by default will save the list to __kim_models_file__ located in the settings directory. The list can be deleted using __unset_kim_models()__.
- Specifying a default __kim_api_directory__ where the kim-api-activate and kim-api-collections-management commands are located for a particular kim-api version.  This allows for potentials to extract the list of currently installed kim models whenever LAMMPS potentials are loaded.  The default value can be set using __set_kim_api_directory()__ and forgotten using __unset_kim_api_directory()__.

__NOTE__ that if both settings are used, the default loading behavior will use the values in the kim_models_file over generating the values using the kim api.  If you want to building the list using the kim api, then call unset_kim_models() to delete the kim_models_file.

## 4. Create local copy of database

The local database gives users the options to work completely offline with their own copy of the database and/or to interact with private user-defined records in the same manner as the publicly released records managed by the Interatomic Potentials Repository project.

There are two recommended methods for copying the database:

1. Using a potentials.Database object, call one of the download methods.
2. Download/clone https://github.com/lmhale99/potentials-library to the local library directory.

### 4.1. Using download methods

The Database class has the following methods:

- download_citations() downloads all Citation records
- download_potentials() downloads all Potential records
- download_lammps_potentials() downloads all potential_LAMMPS records and the associated parameter files
- download_all() calls all three of the above methods at once.

When downloading records through this method, there is the option to only save records that do not already exist.  This allows for local modifications to be made to the records, but prevents updates from being noted.

In [15]:
db = potentials.Database(local=True, remote=True)
db.download_citations(verbose=True)

334 citations saved to localpath
 - 334 duplicate citations skipped


### 4.2. Copying from github

The https://github.com/lmhale99/potentials-library repository hosts copies of all records stored in the potentials.nist.gov database allowing for users to get all in one bulk download, perform incremental updates, and manage their own records with version control.  Obtaining the records in this manner has many benefits, such as quicker/easier downloads, all record styles in the database besides the three handled by the potentials package, and more direct control for users, but is not yet guaranteed to be 100% up to date with the database itself.